**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from collections import deque
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Dropout, Flatten

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function act represents the decision process of our algorithm. Two states exist, one when we are training our agent and one when we want to exploit the training results.
>
> When training, two seperate methods are available. By pursuing exploration, the agent randomly picks an action and stores the result of this random decision. By pursuing exploitation, the agent picks a decision based on the data available regarding the reward from each action available at its current state.
>
> When ```epsilon``` is equal to 0, the agent is only using the exploitation method, when it's equal to 1, the agent is only using the exploration method. Changing the value of ```epsilon``` strikes a balance between exploration and exploitation. The exploration method is then chosen at the ```epsilon``` rate.
>
> When the action function is called not under training, only the exploitation method is available.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [13]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The array ```position``` represents the board game on which the rat is moving. It inlcudes a 2 elements board all around it that are marked by a -1. The position on the board are marked by a 0 except for the current position of the rat marked by a 1 when the act function is used. This array keeps track of the position of the rat.
>
> The array ```board``` entails the reward values for each position on the board. The pieces of cheese and poisonous cells are drawn following a binomial law and the ```board``` changes everytime the reset function is called.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    
    # Number of won games
    score = 0
    loss = 0
        
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epoch))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 14.0/16.0. Average score (-2.0)
Win/lose count 7.0/14.0. Average score (-4.5)
Win/lose count 12.0/19.0. Average score (-5.333333333333333)
Win/lose count 8.5/12.0. Average score (-4.875)
Win/lose count 7.5/10.0. Average score (-4.4)
Win/lose count 10.5/12.0. Average score (-3.9166666666666665)
Win/lose count 12.5/17.0. Average score (-4.0)
Win/lose count 13.0/13.0. Average score (-3.5)
Win/lose count 12.5/17.0. Average score (-3.611111111111111)
Win/lose count 13.0/15.0. Average score (-3.45)
Win/lose count 9.0/14.0. Average score (-3.590909090909091)
Win/lose count 10.0/18.0. Average score (-3.9583333333333335)
Win/lose count 10.5/13.0. Average score (-3.8461538461538463)
Win/lose count 7.5/9.0. Average score (-3.6785714285714284)
Win/lose count 10.5/16.0. Average score (-3.8)
Win/lose count 12.0/17.0. Average score (-3.875)
Win/lose count 10.0/9.0. Average score (-3.588235294117647)
Win/lose count 10.0/16.0. Average score (-3.7222222222222223)
Win/lose count 12.0/16.0.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




>We know
>\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
>
>By using the Markov property and defining the function as a Bellman equation we have that:
>
>\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
>
>One assumes that the optimal policy $\pi^*$ exists, the optimal Q function is defined as:
>
>\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> 
\end{equation*}
>
>\begin{equation*}
Q^*(s,a)= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \> 
\end{equation*}
>
>\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')] \> 
\end{equation*}
>
>\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
>
>Finally, a plausible objective is to minimize the loss function defined as:
>
>\begin{equation*}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}( ( Q^* (s,a,\theta ) - Q(s,a,\theta)) ^2) 
\end{equation*}
>
>\begin{equation*}
\mathcal{L}(\theta) =  E_{s' \sim \pi^*(.|s,a)} ( ( r(s,a)+\gamma\max_{a'}Q^{*}(s',a', \theta) - Q(s,a,\theta)) ^2) 
\end{equation*}
>
>\begin{equation*}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        # Instead of list, we use deque. The deque type works like a list but conveniently handles maximum length
        # in a FIFO order.
        self.memory = deque(maxlen=self.max_memory)

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        temp = np.random.randint(0, len(self.memory))
        return self.memory[temp]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(max_memory = memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
    

    def learned_act(self, s):
        
        s = np.reshape(s,[1,5,5,self.n_state])
        # Predict the reward value based on the given state
        act_values = self.model.predict(s)
        return np.argmax(act_values[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        s_ = np.reshape(s_,[1,5,5,self.n_state])
        n_s_ = np.reshape(n_s_,[1,5,5,self.n_state])
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            
            # Initialise the sequence of the minibatch
            seq = self.memory.random_access()
            
            s_= seq[0]
            n_s_= seq[1]
            a_ = seq[2]
            r_= seq[3]
            game_over_= seq[4]
            
            input_states[i]= s_
            
            if game_over_:
                target= r_

            else:               
                target = (r_ + self.discount * np.amax(self.model.predict(n_s_)[0]))
                
            target_f = self.model.predict(s_)
            target_f[0][a_] = target
            self.model.fit(s_, target_f, epochs=1, verbose=0)
            
            target_q[i]=target_f
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            

class DQN_FC(DQN):
    def __init__(self, lr=0.1,  *args, **kwargs):
        super(DQN_FC, self).__init__(*args,**kwargs)
        
        # NN Model
        state_size = 50
        
        model = Sequential()
        model.add(Reshape((state_size,), input_shape=(5,5,self.n_state)))
        model.add(Dense(512, input_dim=state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=lr))
        self.model = model

In [18]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=0.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 128)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/020 | Loss 725.6783 | Win/lose count 4.5/2.0 (2.5)
Epoch 001/020 | Loss 8.8614 | Win/lose count 10.0/14.0 (-4.0)
Epoch 002/020 | Loss 1.5302 | Win/lose count 5.0/15.0 (-10.0)
Epoch 003/020 | Loss 1.2406 | Win/lose count 9.5/13.0 (-3.5)
Epoch 004/020 | Loss 1.5038 | Win/lose count 12.5/26.0 (-13.5)
Epoch 005/020 | Loss 4.4571 | Win/lose count 6.5/17.0 (-10.5)
Epoch 006/020 | Loss 3.5876 | Win/lose count 13.0/18.0 (-5.0)
Epoch 007/020 | Loss 1.9641 | Win/lose count 6.5/13.0 (-6.5)
Epoch 008/020 | Loss 2.6022 | Win/lose count 14.5/13.0 (1.5)
Epoch 009/020 | Loss 1.7267 | Win/lose count 6.5/5.0 (1.5)
Epoch 010/020 | Loss 3.2359 | Win/lose count 7.0/20.0 (-13.0)
Epoch 011/020 | Loss 4.5129 | Win/lose count 9.0/13.0 (-4.0)
Epoch 012/020 | Loss 3.3874 | Win/lose count 11.0/13.0 (-2.0)
Epoch 013/020 | Loss 0.7574 | Win/lose count 8.0/19.0 (-11.0)
Epoch 014/020 | Loss 9.0795 | Win/lose count 8.5/16.0 (-7.5)
Epoch 015/020 | Loss 2.3953 | Win/lose count 5.5/6.0 (-0.5)
Epoch 016/020 | Lo

> Now that we have a running DQN, we will investigate the impact of the learning rate, the batch size and the memory size parameters.

In [47]:
lr = [0.2, 0.15, 0.1, 0.05, 0.01]
for i in lr:
    print("The current learning rate assessed is:", i)
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent = DQN_FC(lr=i, grid_size=size, epsilon = 0.1, memory_size=100, batch_size = 32)
    train(agent, env, epochs_train, prefix='fc_train')

The current learning rate assessed is: 0.2
Epoch 000/005 | Loss 2308351.0000 | Win/lose count 4.0/12.0 (-8.0)
Epoch 001/005 | Loss 1921.7947 | Win/lose count 6.0/6.0 (0.0)
Epoch 002/005 | Loss 4978.6270 | Win/lose count 5.5/9.0 (-3.5)
Epoch 003/005 | Loss 1517.0422 | Win/lose count 3.5/8.0 (-4.5)
Epoch 004/005 | Loss 546.9557 | Win/lose count 3.5/4.0 (-0.5)
Final score: -3.3
The current learning rate assessed is: 0.15
Epoch 000/005 | Loss 9.9596 | Win/lose count 2.0/6.0 (-4.0)
Epoch 001/005 | Loss 72.2595 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/005 | Loss 0.1422 | Win/lose count 4.5/6.0 (-1.5)
Epoch 003/005 | Loss 0.6562 | Win/lose count 5.5/13.0 (-7.5)
Epoch 004/005 | Loss 0.2044 | Win/lose count 8.0/10.0 (-2.0)
Final score: -3.1
The current learning rate assessed is: 0.1
Epoch 000/005 | Loss 5427.4463 | Win/lose count 5.5/9.0 (-3.5)
Epoch 001/005 | Loss 529.0611 | Win/lose count 2.5/2.0 (0.5)
Epoch 002/005 | Loss 2.5027 | Win/lose count 1.5/2.0 (-0.5)
Epoch 003/005 | Loss 2.1499 | 

> We will keep the learning rate of lr = 0.1, showing the best Finals score. 

In [49]:
bs = [16, 32, 64, 128]
for i in bs:
    print("The current batch size assessed is:", i)
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent = DQN_FC(lr=0.1, grid_size=size, epsilon = 0.1, memory_size=100, batch_size = i)
    train(agent, env, epochs_train, prefix='fc_train')

The current batch size assessed is: 16
Epoch 000/005 | Loss 0.9763 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/005 | Loss 140.3693 | Win/lose count 4.5/5.0 (-0.5)
Epoch 002/005 | Loss 293.6937 | Win/lose count 1.5/3.0 (-1.5)
Epoch 003/005 | Loss 219.5443 | Win/lose count 2.5/1.0 (1.5)
Epoch 004/005 | Loss 148.3887 | Win/lose count 1.0/4.0 (-3.0)
Final score: -0.7
The current batch size assessed is: 32
Epoch 000/005 | Loss 72.5068 | Win/lose count 2.5/6.0 (-3.5)
Epoch 001/005 | Loss 43.7292 | Win/lose count 1.0/1.0 (0.0)
Epoch 002/005 | Loss 1.7083 | Win/lose count 9.0/14.0 (-5.0)
Epoch 003/005 | Loss 0.2602 | Win/lose count 2.5/7.0 (-4.5)
Epoch 004/005 | Loss 0.6872 | Win/lose count 10.0/12.0 (-2.0)
Final score: -3.0
The current batch size assessed is: 64
Epoch 000/005 | Loss 221.2805 | Win/lose count 5.0/6.0 (-1.0)
Epoch 001/005 | Loss 0.0934 | Win/lose count 5.5/6.0 (-0.5)
Epoch 002/005 | Loss 0.2631 | Win/lose count 8.5/14.0 (-5.5)
Epoch 003/005 | Loss 1.5359 | Win/lose count 5.5/15.0 

> We will keep the batch size batch_size = 128.

In [50]:
mem = [50, 100, 1000, 5000]
for i in mem:
    print("The current memory size assessed is:", i)
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent = DQN_FC(lr=0.1, grid_size=size, epsilon = 0.1, memory_size=i, batch_size = 128)
    train(agent, env, epochs_train, prefix='fc_train')

The current memory size assessed is: 50
Epoch 000/005 | Loss 36800606208.0000 | Win/lose count 5.0/1.0 (4.0)
Epoch 001/005 | Loss 7898934784.0000 | Win/lose count 4.5/4.0 (0.5)
Epoch 002/005 | Loss 60547194880.0000 | Win/lose count 5.0/2.0 (3.0)
Epoch 003/005 | Loss 4635257856.0000 | Win/lose count 4.0/2.0 (2.0)
Epoch 004/005 | Loss 42785177600.0000 | Win/lose count 5.5/8.0 (-2.5)
Final score: 1.4
The current memory size assessed is: 100
Epoch 000/005 | Loss 6536096.5000 | Win/lose count 4.5/7.0 (-2.5)
Epoch 001/005 | Loss 655.4286 | Win/lose count 3.5/7.0 (-3.5)
Epoch 002/005 | Loss 0.3590 | Win/lose count 8.0/17.0 (-9.0)
Epoch 003/005 | Loss 0.1139 | Win/lose count 10.0/19.0 (-9.0)
Epoch 004/005 | Loss 2.7073 | Win/lose count 8.5/18.0 (-9.5)
Final score: -6.7
The current memory size assessed is: 1000
Epoch 000/005 | Loss 5357075456.0000 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/005 | Loss 8704337920.0000 | Win/lose count 1.0/3.0 (-2.0)
Epoch 002/005 | Loss 23717455872.0000 | Win/lose

> We will keep the memory size memory_size = 1000.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [11]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        filters = 32
        kernel_size = (2,2)
        
        model = Sequential()
        model.add(Conv2D(filters, kernel_size, input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(filters, kernel_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/020 | Loss 0.0023 | Win/lose count 2.5/8.0 (-5.5)
Epoch 001/020 | Loss 0.0014 | Win/lose count 2.0/0 (2.0)
Epoch 002/020 | Loss 0.0077 | Win/lose count 5.5/1.0 (4.5)
Epoch 003/020 | Loss 0.0023 | Win/lose count 5.0/1.0 (4.0)
Epoch 004/020 | Loss 0.0072 | Win/lose count 4.5/3.0 (1.5)
Epoch 005/020 | Loss 0.0515 | Win/lose count 10.0/4.0 (6.0)
Epoch 006/020 | Loss 0.0904 | Win/lose count 13.5/3.0 (10.5)
Epoch 007/020 | Loss 0.0777 | Win/lose count 10.5/8.0 (2.5)
Epoch 008/020 | Loss 0.0919 | Win/lose count 17.5/2.0 (15.5)
Epoch 009/020 | Loss 0.0873 | Win/lose count 18.0/4.0 (14.0)
Epoch 010/020 | Loss 0.1815 | Win/lose count 19.0/6.0 (13.0)
Epoch 011/020 | Loss 0.3430 | Win/lose count 24.0/3.0 (21.0)
Epoch 012/020 | Loss 0.4449 | Win/lose count 15.5/4.0 (11.5)
Epoch 013/020 | Loss 0.7719 | Win/lose count 22.5/9.0 (13.5)
Epoch 014/020 | Loss 0.7809 | Win/lose count 18.0/4.0 (14.0)
Epoch 015/020 | Loss 0.2718 | Win/lose count 19.5/8.0 (11.5)
Epoch 016/020 | Loss 0.5369 | Win/los

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [19]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(grid_size=size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/12.0. Average score (-7.0)
Win/lose count 7.5/10.0. Average score (-4.75)
Win/lose count 5.5/10.0. Average score (-4.666666666666667)
Win/lose count 7.5/6.0. Average score (-3.125)
Win/lose count 6.5/8.0. Average score (-2.8)
Win/lose count 8.5/15.0. Average score (-3.4166666666666665)
Win/lose count 6.5/7.0. Average score (-3.0)
Win/lose count 4.5/2.0. Average score (-2.3125)
Win/lose count 5.5/6.0. Average score (-2.111111111111111)
Win/lose count 3.5/4.0. Average score (-1.95)
Win/lose count 11.0/6.0. Average score (-1.3181818181818181)
Win/lose count 10.0/11.0. Average score (-1.2916666666666667)
Win/lose count 7.0/13.0. Average score (-1.6538461538461537)
Win/lose count 3.0/7.0. Average score (-1.8214285714285714)
Win/lose count 9.5/17.0. Average score (-2.2)
Win/lose count 4.5/5.0. Average score (-2.09375)
Win/lose count 3.5/5.0. Average score (-2.0588235294117645)
Win/lose count 3.5/2.0. Average score (-1.8611111111111112)
Win/lose count 7.0/6.

In [20]:
HTML(display_videos('cnn_test10.mp4'))

In [21]:
HTML(display_videos('fc_test10.mp4'))

> We can notice that in both cases, the agent seems to waste time on certain steps going back and forth multiple times. This can be seen as a lack of exploration.
>
> As seen in the environment initialisation, the temperature is used as the p probability distribution. The law followed is a binomial one. So the higher the temperature, the higher the number of cheese and poisonous elements on the board.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [ ]:
def train_explore(agent,env,epoch,prefix=''):
    pass
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        pass
    
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [ ]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

Ressource: Sarah Lina's GitHub

***